In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
.master("local[4]") \
.appName("DF to Disk") \
.config("spark.executor.memory","4g") \
.config("spark.driver.memory","2g") \
.getOrCreate()

adultDataDf = spark.read \
.option("header","True") \
.option("inferSchema","True") \
.option("sep",",") \
.csv("D:\\datasets\\adult.data")

adultTestDf = spark.read \
.option("header","True") \
.option("inferSchema","True") \
.option("sep",",") \
.csv("D:\\datasets\\adult.test")

ModuleNotFoundError: No module named 'findspark'

In [2]:
adult_whole_df = adultDataDf.union(adultTestDf)

In [3]:
adult_whole_df.limit(5).toPandas().head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,output
0,39,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


# Veri Temizliği 

In [4]:
from pyspark.sql.functions import *


In [5]:
adult_whole_df1 = adult_whole_df \
.withColumn("workclass", trim(col("workclass"))) \
.withColumn("education", trim(col("education"))) \
.withColumn("marital_status", trim(col("marital_status"))) \
.withColumn("occupation", trim(col("occupation"))) \
.withColumn("relationship", trim(col("relationship"))) \
.withColumn("race", trim(col("race"))) \
.withColumn("sex", trim(col("sex"))) \
.withColumn("native_country", trim(col("native_country"))) \
.withColumn("output", trim(col("output"))) \

In [7]:
adult_whole_df.count()


48842

In [8]:
adult_whole_df1.count()


48842

In [9]:
adult_whole_df2 = adult_whole_df1 \
.withColumn("output", regexp_replace(col("output"), "<=50K.","<=50K")) \
.withColumn("output", regexp_replace(col("output"), ">50K.",">50K"))

# 3 null kontrolü

In [10]:
adult_whole_df2.columns

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education_num',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country',
 'output']

In [13]:
sayac_for_null = 1
for sutun in adult_whole_df.columns:
    if(adult_whole_df2.filter(col(sutun).isNull()).count()>0):
        print(sayac_for_null,". ", sutun, " içinde null var.")
    else:
        print(sayac_for_null, " " , sutun)
    sayac_for_null+=1

1   age
2   workclass
3   fnlwgt
4   education
5   education_num
6   marital_status
7   occupation
8   relationship
9   race
10   sex
11   capital_gain
12   capital_loss
13   hours_per_week
14   native_country
15   output


# 4 "? Kontrolü"

In [16]:
sayac_for_question = 1
for sutun in adult_whole_df2.columns:
    if(adult_whole_df2.filter(col(sutun).contains("?")).count()>0):
        print(sayac_for_question,". ", sutun, " içinde ? var.")
    else:
        print(sayac_for_question, " " , sutun)
    sayac_for_null+=1

1   age
1 .  workclass  içinde ? var.
1   fnlwgt
1   education
1   education_num
1   marital_status
1 .  occupation  içinde ? var.
1   relationship
1   race
1   sex
1   capital_gain
1   capital_loss
1   hours_per_week
1 .  native_country  içinde ? var.
1   output


In [18]:
adult_whole_df2.select("workclass","occupation","native_country","output")\
.filter(
    col("workclass").contains("?") |
    col("occupation").contains("?") |
    col("native_country").contains("?")
) \
.groupBy("workclass","occupation","native_country","output").count() \
.orderBy(col("count").desc())\
.toPandas().head()

,workclass,occupation,native_country,output,count
0,?,?,United-States,<=50K,2284
1,?,?,United-States,>50K,246
2,Private,Other-service,?,<=50K,100
3,Private,Sales,?,<=50K,55
4,Private,Prof-specialty,?,<=50K,51


In [24]:
adult_whole_df3 = adult_whole_df2 \
.filter(~(col("workclass").contains("?") |
    col("occupation").contains("?") |
    col("native_country").contains("?")
       ))

In [25]:
print(adult_whole_df3.count())
print(adult_whole_df2.count())

45222
48842


# 5. ZAYIF SINIFLARIN KALDIRILMASI

// workclass niteliğinde never-worked ve without-pay sınıfları ve occupation niteliğinde Armed-Forces sınıfı çok az tekrarlanmış. Veri setinden çıkarılabilir.

In [26]:
adult_whole_df4 = adult_whole_df3 \
.filter(~(
    col("workclass").contains("never-worked") | col("workclass").contains("without-pay") | 
       col("occupation").contains("Armed-Forces") | col("native_country").contains("Holand-Netherlands")
))

print(adult_whole_df3.count())
print(adult_whole_df4.count())

45222
45207


# 6. EĞİTİM DURUMUYLA İLGİLİ KATEGORİLERİ BİRLEŞTİRME

// education niteliğindeki:
  1st-4th, 5th-6th, 7th-8th: elementary-school
9th, 10th, 11th, 12th: high-school
Masters, Doctorate: high-education
Bachelors, Some-college: undergraduate
sınıfları yukarıdaki gibi birleştirilebilir.

In [27]:
adult_whole_df5 = adult_whole_df4.withColumn("education_merged",
when(col("education").isin("1st-4th", "5th-6th", "7th-8th"), "Elementary-School") 
.when(col("education").isin("9th", "10th", "11th", "12th"), "High-School") 
.when(col("education").isin("Masters","Doctorate"), "Postgraduate")
.when(col("education").isin("Bachelors","Some-college"), "Undergraduate")
.otherwise(col("education"))
                                            
)

adult_whole_df5.select("education","education_merged").toPandas().head(20)

,education,education_merged
0,Bachelors,Undergraduate
1,Bachelors,Undergraduate
2,HS-grad,HS-grad
3,11th,High-School
4,Bachelors,Undergraduate
5,Masters,Postgraduate
6,9th,High-School
7,HS-grad,HS-grad
8,Masters,Postgraduate
9,Bachelors,Undergraduate


// Sütun sırasını değiştirelim. education_merge'i education yanına alalım



In [28]:
nitelik_siralama = ["workclass", "education", "education_merged", "marital_status", "occupation", "relationship", "race",
     "sex", "native_country", "age", "fnlwgt", "education_num", "capital_gain", "capital_loss", "hours_per_week","output"]

adult_whole_df6 = adult_whole_df5.select(nitelik_siralama)
adult_whole_df6.toPandas().head()

,workclass,education,education_merged,marital_status,occupation,relationship,race,sex,native_country,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,output
0,State-gov,Bachelors,Undergraduate,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,39,77516.0,13.0,2174.0,0.0,40.0,<=50K
1,Self-emp-not-inc,Bachelors,Undergraduate,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,50,83311.0,13.0,0.0,0.0,13.0,<=50K
2,Private,HS-grad,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,38,215646.0,9.0,0.0,0.0,40.0,<=50K
3,Private,11th,High-School,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,53,234721.0,7.0,0.0,0.0,40.0,<=50K
4,Private,Bachelors,Undergraduate,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,28,338409.0,13.0,0.0,0.0,40.0,<=50K


In [30]:
adult_whole_df6 \
.coalesce(1) \
.write \
.mode("overwrite") \
.option("sep",",") \
.option("header","True") \
.csv("D:\\Datasets\\adult_processed_pyspark")